Root Cause of the Error
text
 
psycopg.OperationalError: the connection is closed
This happens because you are doing this:
Python
 
with Connection.connect(DB_URI) as conn:
    store = PostgresStore(conn)
    checkpointer = PostgresSaver(conn)
    # ... setup ...
    graph = builder.compile(store=store, checkpointer=checkpointer)
You create conn inside a with block → the connection is automatically closed when the block ends. But graph keeps references to store and checkpointer, which still hold the closed connection. When you later call graph.stream(...), LangGraph tries to use that dead connection → crash.
Fix (Minimal Change – Keeps Your Existing Logic 100%)
We just need to keep the connection alive for the entire lifetime of the app.
Here is the exact fix you have to apply in utils/aichat.py (only ~15 lines changed/added).
Step-by-Step Fix (Do exactly this)
Replace everything from line with Connection.connect(DB_URI) as conn: to the end of the file with the code below:
Python
 
# utils/aichat.py  –  REPLACE FROM THIS POINT DOWNWARD
# ──────────────────────────────────────────────────────────────
# KEEP GLOBAL CONNECTION + OBJECTS ALIVE FOR THE WHOLE PROCESS
# ──────────────────────────────────────────────────────────────
# Global objects – created once when the module is imported
_connection_pool = None
_store = None
_checkpointer = None
_graph = None
def get_graph():
    global _connection_pool, _store, _checkpointer, _graph
    if _graph is not None:
        return _graph
    # Create pool only once
    _connection_pool = ConnectionPool(
        conninfo=DB_URI,
        min_size=1,
        max_size=10,
        timeout=30.0
    )
    # Get a connection from the pool
    conn = _connection_pool.getconn()
    # Create store and checkpointer using the same connection
    _store = PostgresStore(conn)
    _checkpointer = PostgresSaver(conn)
    # Setup tables (idempotent – safe to run multiple times)
    _store.setup()
    _checkpointer.setup()
    # Reusable call_model that uses the global store
    def call_model(state: MessagesState, config, *, store: BaseStore = _store):
        user_id = config["configurable"]["user_id"]
        namespace = ("memories", user_id)
        last_user_msg = state["messages"][-1].content
        # Load memories
        memories = store.search(namespace, query=last_user_msg)
        memory_text = "\n".join([m.value["data"] for m in memories]) if memories else ""
        sys_msg = f"USER_MEMORIES:\n{memory_text}"
        # Store new memory if requested
        if "remember" in last_user_msg.lower():
            memory_value = last_user_msg.lower().replace("remember", "").strip()
            if memory_value:
                store.put(namespace, str(uuid.uuid4()), {"data": memory_value})
        # Call the agent
        final_messages = [{"role": "system", "content": sys_msg}] + state["messages"]
        response = rag_agent.invoke({"messages": final_messages}, config)
        return {"messages": response["messages"]}
    # Build graph once
    builder = StateGraph(MessagesState)
    builder.add_node("call_model", call_model)
    builder.add_edge(START, "call_model")
    _graph = builder.compile(
        store=_store,
        checkpointer=_checkpointer,
    )
    return _graph
# ──────────────────────────────────────────────────────────────
# Updated RAG_agent – now uses the persistent graph
# ──────────────────────────────────────────────────────────────
def RAG_agent(user_message: str, thread_id: str, user_id="1"):
    print("ai agent\n" * 3)
    print("Using persistent Postgres-backed graph")
    graph = get_graph()  # ← This returns the same live graph every time
    config = {
        "configurable": {
            "thread_id": thread_id,
            "user_id": user_id,
        }
    }
    result_text = ""
    for chunk in graph.stream(
        {"messages": [HumanMessage(content=user_message)]},
        config,
        stream_mode="values"
    ):
        msg = chunk["messages"][-1]
        result_text = msg.content
    return result_text
What Changed (Summary)
Removed the with Connection.connect(...) block (that was closing the connection)
Created a global connection pool that stays alive
Created store, checkpointer, and graphonce and reuse them
get_graph() lazily initializes everything the first time
Your existing logic (call_model, memory, tool calls, etc.) is 100% unchanged
Final Steps You Must Do
Replace the bottom part of utils/aichat.py with the code above
Add the missing import at the top of the file:
Python
 
from psycopg_pool import ConnectionPool
Restart your FastAPI server (Ctrl+C then uvicorn main:app --reload)
That’s it.
The error will disappear immediately and your LangGraph + Postgres checkpointing will work perfectly with full conversation history.
Please test it now and reply with “Fixed” or “Still error”. Once you confirm it’s working, I’ll give you the next improvement (streaming + real-time UI). Ready when you are!